In [ ]:
import pandas as pd
print(pd.__version__)
from utils.utils import *
from utils.openai_utils import *


In [ ]:
from utils.perturbation_utils import *

## Start here

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
import pickle
from utils.openai_utils import *
client = OpenAI()




In [ ]:
name = 'Batch_perturbation_severe'
dataset = 'CSII'

prompt_type = 'Metric'

model ='gpt-3.5-turbo-0125'

strings = generate_tir_dict_severehypo()

individual_data = load_data_metrics_for_prompt_short(dataset) 
prompt_list = {}
for (hypo, hyper,tir), description in strings.items():
    for id in individual_data:

        pat = individual_data[id]
        comparator = description

        prompt_list[(hypo, hyper,tir,id)] = instructions(comparator,pat)


for model in [model]:

    batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"
    os.makedirs(batch_dir, exist_ok=True)
    
   
    batch_json = make_batch_json_openai_perturbation(prompt_list, batch_dir , model)

    client = OpenAI()
    batch_input_file = client.files.create(
        file=open(batch_json, "rb"),
        purpose="batch"
    )
    save_input_file_openai(batch_input_file, batch_dir, batch_json)
    

    client = OpenAI()
    batch_input_file_id = batch_input_file.id
    batch_sent= client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/responses",
        completion_window="24h",
        metadata={
            "description": f"{model}_{dataset}_{datetime.now().strftime('%Y_%m_%d_%Hh')}"
        }
    )
    save_batch_file_openai(batch_sent, batch_dir)

In [ ]:
prompt_type = 'Metric'

name = "Batch_perturbation_severe"
dataset = 'CSII'

batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"
os.makedirs(batch_dir, exist_ok=True)
    

In [ ]:
models =["o4-mini-2025-04-16"]
models = [model]
for model in models:
    batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"
    batch_metadata = load_batch_file_openai(batch_dir)
    batch =client.batches.retrieve(batch_metadata['batch_id'])
    print(batch)
    save_batch_file_openai(batch, batch_dir)
